# 2. 수집 데이터 전처리

### 2-1. 웹크롤링 후 저장된 파일 중 누락된 파일이 있는지 확인

In [ ]:
'''import pandas as pd
import datetime
import os

ds1 = pd.date_range(start='20170101', end='20220701', freq='H')
ds1 = set(list(map(lambda x: str(x), ds1)))
# print(ds1)

filepath = '{해당 경로 입력}'
files = os.listdir(filepath)

ds2 = []
for i in range(len(files)):

    t = files[i].split('_')[1].split('.')[0]

    Y, m, d, H, M = t[:4], t[4:6], t[6:8], t[8:10], t[10:12]
    d = Y + '-' + m + '-' + d + ' ' + H + ':' + M + ':' + '00'

    ds2.append(d)

ds2 = set(ds2)

print('누락된 파일:', ds1 - ds2)'''

"import pandas as pd\nimport datetime\nimport os\n\nds1 = pd.date_range(start='20170101', end='20220701', freq='H')\nds1 = set(list(map(lambda x: str(x), ds1)))\n# print(ds1)\n\nfilepath = '{해당 경로 입력}'\nfiles = os.listdir(filepath)\n\nds2 = []\nfor i in range(len(files)):\n\n    t = files[i].split('_')[1].split('.')[0]\n\n    Y, m, d, H, M = t[:4], t[4:6], t[6:8], t[8:10], t[10:12]\n    d = Y + '-' + m + '-' + d + ' ' + H + ':' + M + ':' + '00'\n\n    ds2.append(d)\n\nds2 = set(ds2)\n\nprint('누락된 파일:', ds1 - ds2)"

### 2-2 2017-01-01 00:00:00 ~ 2022-06-30 23:00:00 데이터 생성 (편의상 2022-07-01 날짜 포함)

In [ ]:
'''!pip install fastparquet
import pandas as pd
import os
import datetime
from tqdm import tqdm

index_cols = ['TIMESTAMP','awsId','awsName','height']
feature_cols = ['ta','ws1','ws10','hm','ps']

ds1 = pd.date_range(start='20170101', end='20220701', freq='H')
ds1 = set(list(map(lambda x: str(x), ds1)))
# print(ds1)

filepath = '{해당 경로 입력}'
files = os.listdir(filepath)

# 해당 파일 인덱스만 가져와서 리스트에 저장
files_to_concat = []
for i in range(len(files)):

    t = files[i].split('_')[1].split('.')[0]
    Y, m, d, H, M = t[:4], t[4:6], t[6:8], t[8:10], t[10:12]
    d = Y + '-' + m + '-' + d + ' ' + H + ':' + M + ':' + '00'

    if d in ds1:
        files_to_concat.append(i)

# print(files_to_concat)

# 해당 파일 concat으로 합치기 (시간 소요 큼)
list_of_df = []
for j in tqdm(files_to_concat):
    part_df = pd.read_parquet(os.path.join(filepath, files[j]))
    part_df = part_df.set_index(index_cols)
    part_df = part_df[feature_cols]

    # 시간별 데이별 합치기
    list_of_df.append(part_df)

df = pd.concat(list_of_df)
df = df.reset_index()

display(df)

df.to_parquet('df_accum.gzip', index=None,
                engine='fastparquet', compression='gzip')
'''

"!pip install fastparquet\nimport pandas as pd\nimport os\nimport datetime\nfrom tqdm import tqdm\n\nindex_cols = ['TIMESTAMP','awsId','awsName','height']\nfeature_cols = ['ta','ws1','ws10','hm','ps']\n\nds1 = pd.date_range(start='20170101', end='20220701', freq='H')\nds1 = set(list(map(lambda x: str(x), ds1)))\n# print(ds1)\n\nfilepath = '{해당 경로 입력}'\nfiles = os.listdir(filepath)\n\n# 해당 파일 인덱스만 가져와서 리스트에 저장\nfiles_to_concat = []\nfor i in range(len(files)):\n\n    t = files[i].split('_')[1].split('.')[0]\n    Y, m, d, H, M = t[:4], t[4:6], t[6:8], t[8:10], t[10:12]\n    d = Y + '-' + m + '-' + d + ' ' + H + ':' + M + ':' + '00'\n\n    if d in ds1:\n        files_to_concat.append(i)\n\n# print(files_to_concat)\n\n# 해당 파일 concat으로 합치기 (시간 소요 큼)\nlist_of_df = []\nfor j in tqdm(files_to_concat):\n    part_df = pd.read_parquet(os.path.join(filepath, files[j]))\n    part_df = part_df.set_index(index_cols)\n    part_df = part_df[feature_cols]\n\n    # 시간별 데이별 합치기\n    list_of_df.append(part

### 2-3. 초기 파라미터 설정

In [ ]:
index_col = ['TIMESTAMP','awsId','awsName','height']
feature_cols = ['ta','ws1','ws10','hm','ps']
target_col = []

### 2-4. 전처리1:
- 기본 전처리

In [ ]:
import pandas as pd
import os
import gc
import datetime

# 원본 데이터 읽기
raw_df = pd.read_parquet('df_accum.gzip')
# display(raw_df)

df = raw_df.copy()

# 데이터 일부만 가져오기
cols = []
cols.extend(index_col)
cols.extend(feature_cols)
cols.extend(target_col)
df = df[cols]

# 7월 1일 00시 제거
df = df[~(df['TIMESTAMP']=='2022-07-01 00:00')]

display(df)

# 불필요한 메모리 제거
del raw_df
gc.collect()

,TIMESTAMP,awsId,awsName,height,ta,ws1,ws10,hm,ps
0,2017-01-01 00:00,90,속초,18.0,4.1,1.3,1.5,50.0,1026.300049
1,2017-01-01 00:00,92,양양(공),75.0,NaN,NaN,NaN,NaN,NaN
2,2017-01-01 00:00,93,북춘천,96.0,-2.8,0.6,1.2,91.0,1030.400024
3,2017-01-01 00:00,95,철원,155.0,-3.1,0.0,0.3,92.0,1030.500000
4,2017-01-01 00:00,96,독도,96.0,7.5,3.7,3.5,48.0,1026.800049
...,...,...,...,...,...,...,...,...,...
33897266,2022-06-30 23:00,989,안덕화순,90.0,25.6,2.0,2.6,98.0,1014.200012
33897267,2022-06-30 23:00,990,낙천,76.0,25.6,4.0,3.9,92.0,1013.799988
33897268,2022-06-30 23:00,991,옥포,28.0,26.5,2.7,2.1,79.0,1014.299988
33897269,2022-06-30 23:00,992,하빈,36.0,26.6,1.8,2.0,80.0,1013.900024


22

### 2.5 전처리2:
- 분석 대상 관측소 선택 및 관측소별 시계열 데이터 생성

> 대표 지역 5개 무작위 선택

In [ ]:
import json
import os
import pandas

if not os.path.exists('temp'):
    os.mkdir('temp')

feature_cols = ['ta','ws1','ws10','hm','ps']
new_df = df.copy()

# 분석 대상 관측소 선택
with open('awsId_awsNames.json','r') as json_file:
    f = json.load(json_file)

places = ['서울','대전','대구', '부산', '광주']

dict_ = { v: k for k, v in f.items() }

place_ids = list(map(lambda x: dict_.get(x), places))
# print(place_ids)

# 불필요한 변수 제거
df = df.drop(columns=['awsName','height'])

# 해당 5개 지역에 해당하는 데이터만 가져오기 (서울, 대전, 대구, 부산, 광주)
for place_id in place_ids:
    print(place_id, '전처리 시작.')
    tempdf1 = df.copy()
    tempdf1 = tempdf1[tempdf1['awsId'].isin([place_id])].reset_index(drop=True)
    tempdf2 = tempdf1.copy()
    tempdf2 = tempdf2[['TIMESTAMP']].set_index('TIMESTAMP')

    # 시간축을 Index로 하고 관측소는 Column으로 늘어뜨리기
    for feature_col in feature_cols:
        df_transpose = tempdf1.copy()
        df_transpose = pd.pivot_table(data=df_transpose, index='TIMESTAMP', columns='awsId', values=feature_col)
        df_transpose.columns = [feature_col + '-' + str(col) for col in df_transpose.columns.values]
        tempdf2 = pd.merge(tempdf2, df_transpose, left_index=True, right_index=True, how='inner')

    new_df = tempdf2
    new_df = new_df.reset_index()

    # 선형보간법으로 결측값 처리
    for col in new_df.columns:
        new_df[col] = new_df[col].interpolate(method='linear')
    print('결측치 처리 완료.')
    print(new_df.isnull().sum())

    # 년월일시 컬럼 생성
    index_col = ['TIMESTAMP','YEAR','MONTH','DAY','TIME']
    feature_cols = ['ta','ws1','ws10','hm','ps']
    target_col = [f'ta-{place_id}']

    new_df['YEAR'] = new_df['TIMESTAMP'].str.slice(0,4).astype(int)
    new_df['MONTH'] = new_df['TIMESTAMP'].str.slice(5,7).astype(int)
    new_df['DAY'] = new_df['TIMESTAMP'].str.slice(9,11).astype(int)
    new_df['TIME'] = new_df['TIMESTAMP'].str.slice(11,13).astype(int)

    new_df['TIMESTAMP'] = pd.to_datetime(new_df['TIMESTAMP'])
    new_df = new_df.set_index(index_col)

    new_df.to_csv('./temp/' + f'df_{place_id}.csv')
    print(place_id, '전처리 완료.')


# 불필요한 메모리 제거
del df
gc.collect()

108 전처리 시작.
결측치 처리 완료.
TIMESTAMP    0
ta-108       0
ws1-108      0
ws10-108     0
hm-108       0
ps-108       0
dtype: int64
108 전처리 완료.
133 전처리 시작.
결측치 처리 완료.
TIMESTAMP    0
ta-133       0
ws1-133      0
ws10-133     0
hm-133       0
ps-133       0
dtype: int64
133 전처리 완료.
143 전처리 시작.
결측치 처리 완료.
TIMESTAMP    0
ta-143       0
ws1-143      0
ws10-143     0
hm-143       0
ps-143       0
dtype: int64
143 전처리 완료.
159 전처리 시작.
결측치 처리 완료.
TIMESTAMP    0
ta-159       0
ws1-159      0
ws10-159     0
hm-159       0
ps-159       0
dtype: int64
159 전처리 완료.
156 전처리 시작.
결측치 처리 완료.
TIMESTAMP    0
ta-156       0
ws1-156      0
ws10-156     0
hm-156       0
ps-156       0
dtype: int64
156 전처리 완료.


0

### 2.6 Feature Engineering
- X, y 생성, 파생변수 생성, Scaling, Train/Test 분할

In [ ]:
def build_X_y(df, base_target, change_target):
    # X, y 생성
    import pandas as pd
    X = df.copy()
    X = X
    y = X[[base_target]]

    # 일별최고기온, 일별최저기온 변수 생성
    y[base_target + '-day-min'] = y[base_target].rolling(24).min().shift(1).interpolate().astype('float32')
    y[base_target + '-day-max'] = y[base_target].rolling(24).max().shift(1).interpolate().astype('float32')

    # 해당 타겟변수 설정
    y = y[[change_target]]

    return X, y

def add_features(X, lags):
    # Lag 변수 생성
    import pandas as pd
    cols = X.columns.tolist()

    for col in cols:
        for i in range(1,lags+1):
            X[col+'-'+'lag'+str(i)] = X[col].shift(i)

    # X셋에 y와 같은 변수 제거
    for col in X.columns:
        if col in y.columns:
            del X[col]
            # print(X[col], '컬럼이 삭제됐습니다.')

    # Lag 변수 생성 시 생긴 Null값 0 처리
    # X = X.fillna(0)

    return X

def scaling(X):
    # Standard Scaling
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
    return X


def split(X, y, cutoff):
    # Train/Test 데이터 분할
    import pandas as pd
    X = X.reset_index()
    y = y.reset_index()

    X = X.fillna(0)
    y = y.fillna(0)

    X_train, y_train = X[X['TIMESTAMP'] < cutoff], y[y['TIMESTAMP'] < cutoff]  # 2017-01-01 00:00:00 ~ 2022-12-31 23:00:00
    X_test, y_test = X[X['TIMESTAMP'] >= cutoff], y[y['TIMESTAMP'] >= cutoff] # 2022-01-01 00:00:00 ~ 2022-06-30 23:00:00

    X_train, y_train = X_train.set_index(index_col), y_train.set_index(index_col)
    X_test, y_test = X_test.set_index(index_col), y_test.set_index(index_col)

    return X_train, X_test, y_train, y_test

# 3. 기온 예측 모델 개발

### 3-1. Neural Network
> 신경망 계열 모델 사용 (GRU, LSTM)
- GRU를 사용한 이유는 LSTM의 상위호환이고 LSTM의 복잡한 Cell State를 간소화시킨 구조임
- LSTM은 참고용으로 포함시킴

#### 3-1-1. GRU

In [ ]:
# Keras 기반 모델 Hyperparameter Tuning 자동화 라이브러리
!pip install -q -U keras-tuner

In [ ]:
def to_array(X_train, y_train):
    import pandas as pd
    X_train_arr = X_train.to_numpy().reshape(-1, X_train.shape[1], 1)
    y_train_arr = y_train.to_numpy().reshape(-1, 1)
    return X_train_arr, y_train_arr

def set_seed(seed):
    import tensorflow as tf
    import numpy as np
    import random
    import os
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

def model_builder(hp):
    model = tf.keras.models.Sequential()
    model.add(GRU(24, input_shape=(X_train_arr.shape[1], X_train_arr.shape[2]), activation='relu'))
    hp_units = hp.Int('units', min_value=8, max_value=24, step=8)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(1, activation='linear'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.005, 0.001])
    model.compile(loss='mae', optimizer=Adam(learning_rate=hp_learning_rate))

    return model


import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GRU
import keras_tuner as kt
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
import os
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.size'] = 11


# 'pic' 폴더 생성
if not os.path.exists('pic'):
    os.mkdir('pic')

name = 'GRU'
idx = 1

# 지역별 타겟별 Train, Test 데이터셋 생성
for place_id in place_ids:
    print(place_id)
    new_df = pd.read_csv('./temp/' + f'df_{place_id}.csv')
    new_df['TIMESTAMP'] = pd.to_datetime(new_df['TIMESTAMP'])
    new_df = new_df.set_index(index_col)

    target_list = [f'ta-{place_id}',f'ta-{place_id}-day-min',f'ta-{place_id}-day-max']
    for change_target in target_list:
        print('-'*120)
        print(idx)
        base_target = f'ta-{place_id}'
        change_target = change_target
        lags = 4
        cutoff = '2022-01-01 00:00'

        X, y = build_X_y(new_df, base_target, change_target)
        target_col = y.columns.tolist()[0]
        X = add_features(X, lags)
        X = scaling(X)
        X_train, X_test, y_train, y_test = split(X, y, cutoff)
        # print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        # display(X_train.head(2))
        # display(y_train.head(2))

        # 신경망모델 Input용 X_train, y_train 생성
        X_train_arr, y_train_arr = to_array(X_train, y_train)
        X_test_arr, y_test_arr = to_array(X_test, y_test)
        print(X_train_arr.shape, y_train_arr.shape)
        print(X_test_arr.shape, y_test_arr.shape)

        # 시드 고정
        set_seed(99)

        # Initial Parameters
        epochs = 30
        batch_size = 150

        # 모델 하이퍼파라미터 튜닝
        start = time.time()
        print('모델 학습 시작.')
        tuner = kt.Hyperband(hypermodel=model_builder,
                            objective=kt.Objective('loss', direction='min'),
                            max_epochs=10,
                            directory='my_dir',
                            project_name='test')

        tuner.search(X_train_arr, y_train_arr,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test_arr, y_test_arr))

        # 최적화된 모델로 학습
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print('units:', best_hps.get('units'))
        print('learning_rate:', best_hps.get('learning_rate'))

        model = tuner.hypermodel.build(best_hps)

        history = model.fit(X_train_arr, y_train_arr,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(X_test_arr, y_test_arr), verbose=0
                            )

        print('모델 학습 완료.', round(time.time() - start,3), '초 소요')

        y_pred_arr = model.predict(X_test_arr)
        print('mae:', mean_absolute_error(y_test_arr, y_pred_arr))

        # 모델 성능결과 저장 (모델 학습이 시간 소요가 크므로 각 모델 학습완료 후 바로 결과 누적)
        train_start, train_stop = str(y_train.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_train.reset_index().tail(1)['TIMESTAMP'].tolist()[0])
        test_start, test_stop = str(y_test.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_test.reset_index().tail(1)['TIMESTAMP'].tolist()[0])

        result_df = pd.DataFrame({
                        'idx': [idx],
                        'train_start': [train_start],
                        'train_stop': [train_stop],
                        'test_start': [test_start],
                        'test_stop': [test_stop],
                        'place_id': [place_id],
                        'target': [change_target],
                        'model': [name],
                        'MAE': [mean_absolute_error(y_test_arr, y_pred_arr)],
                    })

        filename = 'model-result-test.csv'
        if not os.path.exists(filename):
            result_df.to_csv(filename, index=False)
        else:
            result_df.to_csv(filename, index=False, mode='a', header=None)

        # 예측 기간만 비교 (시각화)
        y_test = pd.DataFrame(y_test_arr.flatten(), index=X_test.reset_index()['TIMESTAMP'])
        y_pred = pd.DataFrame(y_pred_arr.flatten(), index=X_test.reset_index()['TIMESTAMP'])
        y_test.columns = [target_col]
        y_pred.columns = [target_col]

        fig, ax = plt.subplots(nrows=1, figsize=(14,6))
        ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
        ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], '--', color='r', label='y_pred', lw=1)

        ax.set_ylabel(target_col)
        ax.legend()
        ax.grid(True)
        ax.xaxis.set_major_locator(mdates.MonthLocator())

        filename = f'test-pred-{idx}.png'
        fig.savefig(f'./pic/' + filename)

        # 전체 기간 비교
        y_train = pd.DataFrame(y_train_arr.flatten(), index=X_train.reset_index()['TIMESTAMP'])
        y_train.columns = [target_col]

        fig, ax = plt.subplots(nrows=1, figsize=(14,6))

        ax.plot(y_train.reset_index()['TIMESTAMP'], y_train[target_col], label='y_train', color='k', lw=1)
        ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
        ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], linestyle='--', color='r', label='y_pred', lw=1)
        ax.axvline(y_train.tail(1).index.tolist()[0], linestyle='--', color='blue', lw=1)
        ax.axvline(y_test.head(1).index.tolist()[0], linestyle='--', color='blue', lw=1)

        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.tick_params(axis='x', rotation=90)
        ax.set_ylabel(target_col)
        ax.legend()
        ax.grid(True)

        filename = f'test-pred-whole-{idx}.png'
        fig.savefig('./pic/' + filename)
        plt.close()

        idx += 1


Output hidden; open in https://colab.research.google.com to view.

#### 3-1-2. LSTM

In [ ]:
def to_array(X_train, y_train):
    import pandas as pd
    X_train_arr = X_train.to_numpy().reshape(-1, X_train.shape[1], 1)
    y_train_arr = y_train.to_numpy().reshape(-1, 1)
    return X_train_arr, y_train_arr

def set_seed(seed):
    import tensorflow as tf
    import numpy as np
    import random
    import os
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

def model_builder(hp):
    model = tf.keras.models.Sequential()
    model.add(LSTM(24, input_shape=(X_train_arr.shape[1], X_train_arr.shape[2]), activation='relu'))
    hp_units = hp.Int('units', min_value=8, max_value=24, step=8)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(1, activation='linear'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.005, 0.001])
    model.compile(loss='mae', optimizer=Adam(learning_rate=hp_learning_rate))

    return model

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
import keras_tuner as kt
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
import os
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.size'] = 11

# 'pic' 폴더 생성
if not os.path.exists('pic'):
    os.mkdir('pic')

name = 'LSTM'
idx = idx

# 지역별 타겟별 Train, Test 데이터셋 생성
for place_id in place_ids:
    print(place_id)
    new_df = pd.read_csv('./temp/' + f'df_{place_id}.csv')
    new_df['TIMESTAMP'] = pd.to_datetime(new_df['TIMESTAMP'])
    new_df = new_df.set_index(index_col)

    target_list = [f'ta-{place_id}',f'ta-{place_id}-day-min',f'ta-{place_id}-day-max']
    for change_target in target_list:
        print('-'*120)
        print(idx)
        base_target = f'ta-{place_id}'
        change_target = change_target
        lags = 4
        cutoff = '2022-01-01 00:00'

        X, y = build_X_y(new_df, base_target, change_target)
        target_col = y.columns.tolist()[0]
        X = add_features(X, lags)
        X = scaling(X)
        X_train, X_test, y_train, y_test = split(X, y, cutoff)
        # print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        # display(X_train.head(2))
        # display(y_train.head(2))

        # 신경망모델 Input용 X_train, y_train 생성
        X_train_arr, y_train_arr = to_array(X_train, y_train)
        X_test_arr, y_test_arr = to_array(X_test, y_test)
        print(X_train_arr.shape, y_train_arr.shape)
        print(X_test_arr.shape, y_test_arr.shape)

        # 시드 고정
        set_seed(99)

        # Initial Parameters
        epochs = 30
        batch_size = 150

        # 모델 하이퍼파라미터 튜닝
        start = time.time()
        print('모델 학습 시작.')
        tuner = kt.Hyperband(hypermodel=model_builder,
                            objective=kt.Objective('loss', direction='min'),
                            max_epochs=10,
                            directory='my_dir',
                            project_name='test')

        tuner.search(X_train_arr, y_train_arr,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test_arr, y_test_arr))

        # 최적화된 모델로 학습
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print('units:', best_hps.get('units'))
        print('learning_rate:', best_hps.get('learning_rate'))

        model = tuner.hypermodel.build(best_hps)

        history = model.fit(X_train_arr, y_train_arr,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(X_test_arr, y_test_arr), verbose=0
                            )

        print('모델 학습 완료.', round(time.time() - start,3), '초 소요')

        y_pred_arr = model.predict(X_test_arr)
        print('mae:', mean_absolute_error(y_test_arr, y_pred_arr))

        # 모델 성능결과 저장 (모델 학습이 시간 소요가 크므로 각 모델 학습완료 후 바로 결과 누적)
        train_start, train_stop = str(y_train.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_train.reset_index().tail(1)['TIMESTAMP'].tolist()[0])
        test_start, test_stop = str(y_test.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_test.reset_index().tail(1)['TIMESTAMP'].tolist()[0])

        result_df = pd.DataFrame({
                        'idx': [idx],
                        'train_start': [train_start],
                        'train_stop': [train_stop],
                        'test_start': [test_start],
                        'test_stop': [test_stop],
                        'place_id': [place_id],
                        'target': [change_target],
                        'model': [name],
                        'MAE': [mean_absolute_error(y_test_arr, y_pred_arr)],
                    })

        filename = 'model-result-test.csv'
        if not os.path.exists(filename):
            result_df.to_csv(filename, index=False)
        else:
            result_df.to_csv(filename, index=False, mode='a', header=None)

        # 예측 기간만 비교 (시각화)
        y_test = pd.DataFrame(y_test_arr.flatten(), index=X_test.reset_index()['TIMESTAMP'])
        y_pred = pd.DataFrame(y_pred_arr.flatten(), index=X_test.reset_index()['TIMESTAMP'])
        y_test.columns = [target_col]
        y_pred.columns = [target_col]

        fig, ax = plt.subplots(nrows=1, figsize=(14,6))
        ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
        ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], '--', color='r', label='y_pred', lw=1)

        ax.set_ylabel(target_col)
        ax.legend()
        ax.grid(True)
        ax.xaxis.set_major_locator(mdates.MonthLocator())

        filename = f'test-pred-{idx}.png'
        fig.savefig(f'./pic/' + filename)

        # 전체 기간 비교
        y_train = pd.DataFrame(y_train_arr.flatten(), index=X_train.reset_index()['TIMESTAMP'])
        y_train.columns = [target_col]

        fig, ax = plt.subplots(nrows=1, figsize=(14,6))

        ax.plot(y_train.reset_index()['TIMESTAMP'], y_train[target_col], label='y_train', color='k', lw=1)
        ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
        ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], linestyle='--', color='r', label='y_pred', lw=1)
        ax.axvline(y_train.tail(1).index.tolist()[0], linestyle='--', color='blue', lw=1)
        ax.axvline(y_test.head(1).index.tolist()[0], linestyle='--', color='blue', lw=1)

        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.tick_params(axis='x', rotation=90)
        ax.set_ylabel(target_col)
        ax.legend()
        ax.grid(True)

        filename = f'test-pred-whole-{idx}.png'
        fig.savefig('./pic/' + filename)
        plt.close()

        idx += 1


Output hidden; open in https://colab.research.google.com to view.

### 3-2. Ensemble Model (LightGBM, HistGradientBoost)

> Boosting 계열 앙상블 모델 사용 (LightGBM, HistGradientBoosting)
- 공통적으로 위 2개의 모델을 사용한 이유는 모델 학습 속도가 빠르기 때문.
- LightGBM는 XGBoost와 비슷하지만, Level-Wise 방식이 아닌 Leaf-Wise 방식으로 최대손실값(max data loss)에 해당하는 노드만 분기를 해서 시간 소요 단축
- Scikit-Learn 문서에 따르면 HistGradientBoosting의 경우 10,000개 이상의 데이터 샘플이 있을 경우 GBM보다 훨씬 빠름

In [ ]:
import lightgbm
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import time
import joblib

# 'pic' 폴더 생성
if not os.path.exists('pic'):
    os.mkdir('pic')

lgb = lightgbm.LGBMRegressor(objective='regression', n_jobs=4, random_state=99, verbosity=0)
hgb = HistGradientBoostingRegressor(loss='squared_error', early_stopping='auto', random_state=99, verbose=0)

params = {
    'lgb': {
        'max_depth': [5, 10, 20, 50],
        'n_estimators': [50, 80, 100],
        'subsample': [0.8, 1],
        'learning_rate': [0.01, 0.005],
        'num_leaves': [2, 3]
    },
    'hgb': {
        'min_samples_leaf': [2, 3],
        'learning_rate': [0.01, 0.005],
    },
}


idx = idx
models = {lgb:'lgb', hgb:'hgb'}

# 지역별 타겟별 모델별 Train, Test 데이터셋 생성
for place_id in place_ids:
    print(place_id)
    new_df = pd.read_csv('./temp/' + f'df_{place_id}.csv')
    new_df['TIMESTAMP'] = pd.to_datetime(new_df['TIMESTAMP'])
    new_df = new_df.set_index(index_col)

    target_list = [f'ta-{place_id}',f'ta-{place_id}-day-min',f'ta-{place_id}-day-max']
    for change_target in target_list:

        base_target = f'ta-{place_id}'
        change_target = change_target
        lags = 4
        cutoff = '2022-01-01 00:00'

        X, y = build_X_y(new_df, base_target, change_target)
        target_col = y.columns.tolist()[0]
        X = add_features(X, lags)
        X = scaling(X)
        X_train, X_test, y_train, y_test = split(X, y, cutoff)
        print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        # display(X_train.head(2))
        # display(y_train.head(2))
        # display(X_test.head(2))

        for model, name in models.items():
            print('-'*120)
            print(idx)
            start = time.time()
            print(name, 'model 학습 시작합니다.')
            random_search = RandomizedSearchCV(estimator=model, param_distributions=params[name], scoring='neg_mean_absolute_error')
            random_search.fit(X_train, y_train)
            model = random_search.best_estimator_
            model.fit(X_train, y_train)
            print(name, 'model 학습 완료.', round(time.time() - start, 3), '초 소요.')
            y_pred = model.predict(X_test)
            print('mae:', round(mean_absolute_error(y_test, y_pred), 4))

            # 모델 성능결과 저장 (모델 학습이 시간 소요가 크므로 각 모델 학습완료 후 바로 결과 누적)
            train_start, train_stop = str(y_train.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_train.reset_index().tail(1)['TIMESTAMP'].tolist()[0])
            test_start, test_stop = str(y_test.reset_index().head(1)['TIMESTAMP'].tolist()[0]), str(y_test.reset_index().tail(1)['TIMESTAMP'].tolist()[0])

            result_df = pd.DataFrame({
                            'idx': [idx],
                            'train_start': [train_start],
                            'train_stop': [train_stop],
                            'test_start': [test_start],
                            'test_stop': [test_stop],
                            'place_id': [place_id],
                            'target': [change_target],
                            'model': [name],
                            'MAE': [mean_absolute_error(y_test, y_pred)],
                        })

            filename = 'model-result-test.csv'
            if not os.path.exists(filename):
                result_df.to_csv(filename, index=False)
            else:
                result_df.to_csv(filename, index=False, mode='a', header=None)

            import matplotlib.pyplot as plt
            import numpy as np
            import matplotlib.dates as mdates

            plt.rcParams['font.size'] = 11

            # 예측 기간만 비교 (시각화)
            fig, ax = plt.subplots(nrows=1, figsize=(14,6))

            y_pred = pd.DataFrame(y_pred, index=X_test.index, columns=[target_col])

            ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
            ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], '--', color='r', label='y_pred', lw=1)

            if not os.path.exists('pic'):
                os.mkdir('pic')

            filename = f'test-pred-{idx}.png'
            fig.savefig(f'./pic/' + filename)

            # 전체 기간 비교
            fig, ax = plt.subplots(nrows=1, figsize=(14,6))

            ax.plot(y_train.reset_index()['TIMESTAMP'], y_train[target_col], label='y_train', color='k', lw=1)
            ax.plot(y_test.reset_index()['TIMESTAMP'], y_test[target_col], label='y_test', lw=3)
            ax.plot(y_pred.reset_index()['TIMESTAMP'], y_pred[target_col], linestyle='--', color='r', label='y_pred', lw=1)
            ax.axvline(y_train.tail(1).reset_index()['TIMESTAMP'][0], linestyle='--', color='blue', lw=1)
            ax.axvline(y_test.head(1).reset_index()['TIMESTAMP'][0], linestyle='--', color='blue', lw=1)

            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.tick_params(axis='x', rotation=90)
            ax.set_ylabel(target_col[0])
            ax.legend()
            ax.grid(True)

            filename = f'test-pred-whole-{idx}.png'
            fig.savefig('./pic/' + filename)
            plt.close()

            idx += 1


Output hidden; open in https://colab.research.google.com to view.